In [1]:
#.............................................
# IMPORT STATEMENTS
#.............................................

# local env is py310

#Handling data
import numpy as np
import netCDF4 as nc
#Plotting figures
import matplotlib.pyplot as plt #Main plotting package
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import cartopy.mpl.ticker as cticker

#machine learning package
import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow import keras
from keras import layers
from tensorflow.keras import layers, Model

import numpy as np

from tensorflow.keras import layers, Sequential, Input, Model


from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, LeakyReLU, Flatten, Dense, Reshape, TimeDistributed, LSTM, ConvLSTM2D, Cropping2D
from tensorflow.keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

from netCDF4 import Dataset
import os


2.14.0
Num GPUs Available:  0


In [2]:
import numpy as np
from netCDF4 import Dataset
import glob

# Specify the path to the NetCDF files
file_paths = glob.glob("C:/Users/hebam/OneDrive - University of Calgary/Mitacs/data/CMIP6_M1/incl/sc1/*.nc")  # Adjust path as necessary

# Initialize a list to store data
all_data = []

for file_path in file_paths:
    with Dataset(file_path, mode='r') as nc_file:
        # Assuming the variable you need is named 'variable_name'
        # Replace 'variable_name' with the actual variable name in your files
        variable_data = nc_file.variables['pr'][:] * 3600
        all_data.append(variable_data)

# Concatenate data from all files along the time axis
all_data = np.concatenate(all_data, axis=0)  # Adjust axis if needed

# Reshape the data
# Assuming data is originally (total_hours, lat, lon)
# Calculate total_hours and reshape accordingly
total_hours = all_data.shape[0]  # Number of hourly timesteps in all years

# Ensure total_hours is divisible by 24 (for daily reshaping)
if total_hours % 24 != 0:
    raise ValueError("Total number of hours is not divisible by 24.")

# Reshape the data to (total_days, 24, lat, lon, 1)
total_days = total_hours // 24
reshaped_data = all_data.reshape((total_days, 24, 7, 14, 1))

# Verify shape
print("Reshaped data shape:", reshaped_data.shape)


Reshaped data shape: (31411, 24, 7, 14, 1)


In [3]:
# Save or process the reshaped data
# For example, save as a new NetCDF file
output_file = "CNRM-ESM2-1_days_sc1.nc"
with Dataset(output_file, mode='w', format='NETCDF4') as nc_out:
    # Create dimensions
    nc_out.createDimension('days', reshaped_data.shape[0])
    nc_out.createDimension('hours', reshaped_data.shape[1])
    nc_out.createDimension('lat', reshaped_data.shape[2])
    nc_out.createDimension('lon', reshaped_data.shape[3])
    nc_out.createDimension('value', reshaped_data.shape[4])

    # Create variables
    data_var = nc_out.createVariable('reshaped_variable', np.float32, ('days', 'hours', 'lat', 'lon', 'value'))
    data_var[:] = reshaped_data

print(f"Reshaped data saved to {output_file}")

Reshaped data saved to CNRM-ESM2-1_days_sc1.nc


The data for the CMIP6 simulations are stored after reshaping and used as inputs for the downscaling process.

In the following code, you should change:
* The scenario you need to downscale
* The paths to the file
* The model that you want to use for the downscaling

In [3]:
from netCDF4 import Dataset
import numpy as np

# Path to the saved NetCDF file
input_file = "Xtest_data.nc"

# Open the NetCDF file
with Dataset(input_file, mode='r') as nc_in:
    # Access the variable
    Xtest = nc_in.variables['reshaped_variable'][:]
    
    # Optionally, you can check its shape or contents
    print(f"Shape of the loaded data: {Xtest.shape}")
    print(f"Data type: {Xtest.dtype}")
    # Example: print the first time step
    print(f"First entry of the loaded data: {Xtest[0]}")

# The variable 'Xtest' is now a NumPy array


Shape of the loaded data: (6805, 24, 7, 14, 1)
Data type: float32
First entry of the loaded data: [[[[5.79511113e-02]
   [9.03310776e-02]
   [1.59006163e-01]
   ...
   [2.99436152e-01]
   [2.37128645e-01]
   [7.13287620e-03]]

  [[3.62389386e-02]
   [4.47142981e-02]
   [9.37897116e-02]
   ...
   [1.90047652e-01]
   [4.98441875e-01]
   [3.60296331e-02]]

  [[2.04674043e-02]
   [1.69600724e-04]
   [3.54836844e-02]
   ...
   [1.51912972e-01]
   [8.15342367e-01]
   [1.82259768e-01]]

  ...

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [1.99226296e+00]
   [1.76571596e+00]
   [4.70805645e-01]]

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [3.05795312e-01]
   [9.07682776e-02]
   [5.32350503e-02]]

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [4.07633808e-04]
   [2.59352848e-04]
   [0.00000000e+00]]]


 [[[5.55651225e-02]
   [6.48962036e-02]
   [1.62656933e-01]
   ...
   [3.15932363e-01]
   [4.38661903e-01]
   [1.760591

In [5]:

# Path to the saved NetCDF file
input_file = "Ytest_data.nc"
nc_file = Dataset(input_file, mode='r')

# List all variable names
print("Variables in the file:")
print(nc_file.variables.keys())

# Open the NetCDF file
with Dataset(input_file, mode='r') as nc_in:
    # Access the variable
    Ytest = nc_in.variables['reshaped_variable'][:]
    
    # Optionally, you can check its shape or contents
    print(f"Shape of the loaded data: {Ytest.shape}")
    print(f"Data type: {Ytest.dtype}")
    # Example: print the first time step
    print(f"First entry of the loaded data: {Ytest[0]}")

# The variable 'Ytest' is now a NumPy array


Variables in the file:
dict_keys(['reshaped_variable'])
Shape of the loaded data: (6805, 24, 26, 54, 1)
Data type: float32
First entry of the loaded data: [[[[2.33416051e-01]
   [3.39410617e-04]
   [0.00000000e+00]
   ...
   [8.74036551e-03]
   [7.29147636e-04]
   [2.42000286e-04]]

  [[2.87801087e-01]
   [7.94011354e-02]
   [0.00000000e+00]
   ...
   [1.68900974e-02]
   [3.53305903e-03]
   [2.89559644e-03]]

  [[1.67782187e-01]
   [1.14292540e-02]
   [2.33648252e-03]
   ...
   [5.52869812e-02]
   [1.44351404e-02]
   [1.08755135e-03]]

  ...

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [2.19443385e-02]
   [2.34306976e-02]
   [2.75585093e-02]]

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]]

  [[0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]
   ...
   [0.00000000e+00]
   [0.00000000e+00]
   [0.00000000e+00]]]


 [[[2.21127778e-01]
   [4.47697676e-04]
   [0.00000000e+00]
  

In [6]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras.layers import Layer, Input
from tensorflow.keras.models import load_model

# Define the ReflectionPadding2D custom layer
class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        super().__init__(**kwargs)
        self.padding = padding

    def call(self, inputs):
        return tf.pad(
            inputs,
            [[0, 0], [self.padding[0], self.padding[0]], [self.padding[1], self.padding[1]], [0, 0]],
            mode="REFLECT"
        )

# Define the residual_block function
def residual_block(x, filters, kernel_size=3, stride=1, padding='same', l2_reg=0.001):
    from tensorflow.keras import layers
    shortcut = x
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding=padding,
                      kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding=padding,
                      kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=stride, padding=padding,
                                 kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(shortcut)
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

# Define the leaky_relu_with_slope function
def leaky_relu_with_slope(x):
    from tensorflow.keras import layers
    return layers.LeakyReLU(alpha=0.2)(x)

# Load the model with all required custom objects
model = load_model(
    './Outputs/WGAN.keras',
    custom_objects={
        'ReflectionPadding2D': ReflectionPadding2D,
        'residual_block': residual_block,
        'leaky_relu_with_slope': leaky_relu_with_slope
    },
    safe_mode=False
)

# Print the model summary
model.summary()

# Test the loaded model
noise_shape = Xtest.shape[:1] + Xtest.shape[2:]
random_noise = np.random.normal(0, 1, noise_shape)
print(random_noise.shape)

loss,acc = model.evaluate([Xtest,random_noise], Ytest)
print('Restored model, MSE: ', loss)
print('Restored model, MAE: ', acc)

Model: "generator"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 noise_input (InputLayer)    [(None, 7, 14, 1)]           0         []                            
                                                                                                  
 low_res_input (InputLayer)  [(None, 24, 7, 14, 1)]       0         []                            
                                                                                                  
 dense (Dense)               (None, 7, 14, 16)            32        ['noise_input[0][0]']         
                                                                                                  
 time_distributed_1 (TimeDi  (None, 24, 7, 14, 16)        160       ['low_res_input[0][0]']       
 stributed)                                                                               

In [7]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np

# Define the extracted hex colors
hex_colors = ["#ffffff",
"#f7f6f5",
"#ece8e5",
"#d0b5ae",
"#b78b81",
"#a56f67",
"#965c57",
"#864d4b",
"#773f42",
"#693139",
"#592631",
"#4a1c2c",
"#391a3c",
"#32215b",
"#4943a0",
"#5054ae",
"#485486",
"#4c6083",
"#54708d",
"#6a8da3",
"#85aab7",
"#c3b799",
"#edb376",
"#e98a4a",
"#e3662e",
"#d83e1d",
"#c5281a",
"#ac1b1e",
"#961a24",
"#831b2b"]  # Full list from above

# Create a colormap from the hex colors
custom_cmap = LinearSegmentedColormap.from_list("custom_gradient", hex_colors)


In [17]:
# .............................................
# PLOTTING PREDICTIONS BASED ON THE UNET
# .............................................

# Generate predictions and reshape them to match the high-res shape
# Generate noise without the time dimension
noise_input = np.random.normal(0, 1, (reshaped_data.shape[0], reshaped_data.shape[2], reshaped_data.shape[3], reshaped_data.shape[4]))

# Generate predictions
test_predictions = model.predict([reshaped_data, noise_input])

print(test_predictions.shape)

982/982 [==============================] - 1441s 1s/step
(31411, 24, 26, 54, 1)


In [18]:
# Save the testing data portion
# For example, save as a new NetCDF file
output_file = "test_predictionsGonly_sc4.nc"
with Dataset(output_file, mode='w', format='NETCDF4') as nc_out:
    # Create dimensions
    nc_out.createDimension('days', test_predictions.shape[0])
    nc_out.createDimension('hours', test_predictions.shape[1])
    nc_out.createDimension('lat', test_predictions.shape[2])
    nc_out.createDimension('lon', test_predictions.shape[3])
    nc_out.createDimension('value', test_predictions.shape[4])

    # Create variables
    data_var = nc_out.createVariable('pr', np.float32, ('days', 'hours', 'lat', 'lon', 'value'))
    data_var[:] = test_predictions

print(f"data saved to {output_file}")

data saved to test_predictionsGonly_sc4.nc


In [19]:
#save output to NC files to do analysis in R

import os
import numpy as np
from netCDF4 import Dataset

# Define your dimensions
time_dim = 24
lat_dim = 26
lon_dim = 54

# Latitude and longitude arrays (customize as needed)
latitudes = np.linspace(48.25, 48.25 + (lat_dim - 1) * 0.5, lat_dim)
longitudes = np.linspace(-120.75, -120.75 + (lon_dim - 1) * 0.5, lon_dim)

# Directory to save the files
output_dir = "CMIP6_sc4_Gonly"
os.makedirs(output_dir, exist_ok=True)

# Loop through each of the 400 samples in test_predictions
for i in range(test_predictions.shape[0]):
    # File name
    file_name = os.path.join(output_dir, f"sample_{i+1}.nc")
    
    # Create the NetCDF file
    with Dataset(file_name, "w", format="NETCDF4") as ncfile:
        # Create dimensions
        ncfile.createDimension("time", time_dim)
        ncfile.createDimension("lat", lat_dim)
        ncfile.createDimension("lon", lon_dim)
        
        # Create variables
        times = ncfile.createVariable("time", "i4", ("time",))
        lats = ncfile.createVariable("lat", "f4", ("lat",))
        lons = ncfile.createVariable("lon", "f4", ("lon",))
        precipitation = ncfile.createVariable("precipitation", "f4", ("time", "lat", "lon"))
        
        # Add metadata (optional)
        ncfile.description = "Test predictions for precipitation downscaling"
        #ncfile.history = "Created on " + Dataset.date.today().strftime("%Y-%m-%d")
        lats.units = "degrees_north"
        lons.units = "degrees_east"
        precipitation.units = "mm/hr"  # Replace with actual units if different
        
        # Assign values to the variables
        times[:] = np.arange(time_dim)  # Replace with actual time values if needed
        lats[:] = latitudes
        lons[:] = longitudes
        precipitation[:, :, :] = test_predictions[i, :, :, :, 0]  # Extract the sample data
        
    print(f"Saved {file_name}")


Saved CMIP6_sc4_Gonly\sample_1.nc
Saved CMIP6_sc4_Gonly\sample_2.nc
Saved CMIP6_sc4_Gonly\sample_3.nc
Saved CMIP6_sc4_Gonly\sample_4.nc
Saved CMIP6_sc4_Gonly\sample_5.nc
Saved CMIP6_sc4_Gonly\sample_6.nc
Saved CMIP6_sc4_Gonly\sample_7.nc
Saved CMIP6_sc4_Gonly\sample_8.nc
Saved CMIP6_sc4_Gonly\sample_9.nc
Saved CMIP6_sc4_Gonly\sample_10.nc
Saved CMIP6_sc4_Gonly\sample_11.nc
Saved CMIP6_sc4_Gonly\sample_12.nc
Saved CMIP6_sc4_Gonly\sample_13.nc
Saved CMIP6_sc4_Gonly\sample_14.nc
Saved CMIP6_sc4_Gonly\sample_15.nc
Saved CMIP6_sc4_Gonly\sample_16.nc
Saved CMIP6_sc4_Gonly\sample_17.nc
Saved CMIP6_sc4_Gonly\sample_18.nc
Saved CMIP6_sc4_Gonly\sample_19.nc
Saved CMIP6_sc4_Gonly\sample_20.nc
Saved CMIP6_sc4_Gonly\sample_21.nc
Saved CMIP6_sc4_Gonly\sample_22.nc
Saved CMIP6_sc4_Gonly\sample_23.nc
Saved CMIP6_sc4_Gonly\sample_24.nc
Saved CMIP6_sc4_Gonly\sample_25.nc
Saved CMIP6_sc4_Gonly\sample_26.nc
Saved CMIP6_sc4_Gonly\sample_27.nc
Saved CMIP6_sc4_Gonly\sample_28.nc
Saved CMIP6_sc4_Gonly\sample_

In [14]:
#Global max and min for the testing set
vmin = 0.01
vmax = max(reshaped_data[0:9, 0:23, :, :, 0].max(), test_predictions[0:9, 0:23, :, :, 0].max())
print(vmin)
print(vmax)

#vmax = 10


from matplotlib.colors import LogNorm
norm = LogNorm(vmin=vmin, vmax=vmax)

print(test_predictions.shape)
print(reshaped_data.shape)

0.01
1.9923015
(31411, 24, 26, 54, 1)
(31411, 24, 7, 14, 1)


In [16]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os
from matplotlib.ticker import FuncFormatter

# Create a directory to save the plots
output_dir = "plots_CMIP6_sc1_Gonly"
os.makedirs(output_dir, exist_ok=True)

# Define the latitude and longitude limits
specified_lat_min, specified_lat_max = 48, 60
specified_lon_min, specified_lon_max = -121, -95

# Latitude and longitude setup for high-resolution data
num_latitudes_high = test_predictions.shape[2]
num_longitudes_high = test_predictions.shape[3]
lats_high = np.linspace(48.25, 48.25 + (num_latitudes_high - 1) * 0.5, num_latitudes_high)
lons_high = np.linspace(-120.75, -120.75 + (num_longitudes_high - 1) * 0.5, num_longitudes_high)

# Edges for high-resolution data
lat_edges_high = np.linspace(lats_high.min() - 0.25, lats_high.max() + 0.25, num_latitudes_high + 1)
lon_edges_high = np.linspace(lons_high.min() - 0.25, lons_high.max() + 0.25, num_longitudes_high + 1)

# Latitude and longitude setup for low-resolution data
num_latitudes_low = reshaped_data.shape[2]
num_longitudes_low = reshaped_data.shape[3]
lats_low = np.linspace(48.25, 48.25 + (num_latitudes_low - 1) * 2, num_latitudes_low)
lons_low = np.linspace(-120.75, -120.75 + (num_longitudes_low - 1) * 2, num_longitudes_low)

# Edges for low-resolution data
lat_edges_low = np.linspace(lats_low.min() - 1, lats_low.max() + 1, num_latitudes_low + 1)
lon_edges_low = np.linspace(lons_low.min() - 1, lons_low.max() + 1, num_longitudes_low + 1)

# Process and save plots for the first 30 samples
for sample_idx in range(11):
    print(f"Processing sample index: {sample_idx}")

    for time_index in range(reshaped_data.shape[1]):  # Assuming 24 time steps
        # Extract data for the selected sample and time step
        low_res = reshaped_data[sample_idx, time_index, :, :, 0]
        generated = test_predictions[sample_idx, time_index, :, :, 0]

        # Ensure data dimensions match the grid
        assert low_res.shape == (num_latitudes_low, num_longitudes_low), "Mismatch in low-res grid dimensions."
        assert generated.shape == (num_latitudes_high, num_longitudes_high), "Mismatch in high-res grid dimensions."

        # Determine the global min and max for consistent color scaling
        vmin = min(low_res.min(), generated.min())
        vmax = max(low_res.max(), generated.max())
        levels = np.linspace(vmin, vmax, 11)

        # Plotting
        fig, axs = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': ccrs.PlateCarree()})
        extent = [specified_lon_min, specified_lon_max, specified_lat_min, specified_lat_max]

        # Plot low-resolution input (native resolution)
        im0 = axs[0].pcolormesh(lon_edges_low, lat_edges_low, low_res, transform=ccrs.PlateCarree(),
                                cmap=custom_cmap, shading='auto', norm=norm)
        axs[0].set_title(f"Low Resolution (Time {time_index})")
        axs[0].coastlines()
        axs[0].add_feature(cfeature.BORDERS, linestyle=':')
        axs[0].set_extent(extent, crs=ccrs.PlateCarree())
        axs[0].set_xticks(np.linspace(specified_lon_min, specified_lon_max, 5), crs=ccrs.PlateCarree())
        axs[0].set_yticks(np.linspace(specified_lat_min, specified_lat_max, 5), crs=ccrs.PlateCarree())
        axs[0].set_xlabel("Longitude")
        axs[0].set_ylabel("Latitude")

        # Plot generated high-resolution prediction
        im1 = axs[1].pcolormesh(lon_edges_high, lat_edges_high, generated, transform=ccrs.PlateCarree(),
                                cmap=custom_cmap, shading='auto', norm=norm)
        axs[1].set_title(f"Generated Prediction (Time {time_index})")
        axs[1].coastlines()
        axs[1].add_feature(cfeature.BORDERS, linestyle=':')
        axs[1].set_extent(extent, crs=ccrs.PlateCarree())
        axs[1].set_xticks(np.linspace(specified_lon_min, specified_lon_max, 5), crs=ccrs.PlateCarree())
        axs[1].set_yticks(np.linspace(specified_lat_min, specified_lat_max, 5), crs=ccrs.PlateCarree())
        axs[1].set_xlabel("Longitude")
        axs[1].set_ylabel("Latitude")

        # Add a common colorbar below the plots
        cbar_width = 0.6  # Adjust this value to control the width of the colorbar
        cbar_left = (1 - cbar_width) / 2  # Center the colorbar

        cbar_ax = fig.add_axes([cbar_left, 0.15, cbar_width, 0.02])  # Adjust the width and center it
        cbar = fig.colorbar(im1, cax=cbar_ax, orientation='horizontal')
        cbar.set_label("Hourly Precipitation (mm)")

        # Customize the colorbar ticks
        def decimal_formatter(x, pos):
            return f"{x:.2f}"  # Format ticks as decimal with two decimal places

        cbar.formatter = FuncFormatter(decimal_formatter)
        cbar.update_ticks()  # Update the colorbar with new tick formatting

        # Save the plot to a file
        output_file = os.path.join(output_dir, f"sample_{sample_idx}_time_{time_index}.png")
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        plt.close(fig)

    print(f"Sample {sample_idx} processed and saved.")


Processing sample index: 0
Sample 0 processed and saved.
Processing sample index: 1
Sample 1 processed and saved.
Processing sample index: 2
Sample 2 processed and saved.
Processing sample index: 3
Sample 3 processed and saved.
Processing sample index: 4
Sample 4 processed and saved.
Processing sample index: 5
Sample 5 processed and saved.
Processing sample index: 6
Sample 6 processed and saved.
Processing sample index: 7
Sample 7 processed and saved.
Processing sample index: 8
Sample 8 processed and saved.
Processing sample index: 9
Sample 9 processed and saved.
Processing sample index: 10
Sample 10 processed and saved.
